In [12]:
import pandas as pd
import numpy as np
import glob
import rhessys.utilities as rut
import sys
import ProgressBar as pb
import ipyparallel as par
import time

In [13]:
c = par.Client() # select the first two engines

In [14]:
c.ids

[0, 1, 2, 3, 5, 6, 7]

In [15]:
view = c.load_balanced_view()

In [36]:
calname = 'snow_cal10'
files = glob.glob('/RHESSys/jemez/out/%s/*_basin.daily'%(calname))

In [37]:
%%px
calname = 'snow_cal10'
import rhessys.utilities as rut

In [38]:
def process_cal(fl):
    tmp = rut.readRHESSysBasin(fl)
    idx = int(fl.split('_')[-2])
    
    #tmp.to_pickle('/Volumes/data/RHESSys_out/Vcm/%s/%s_%s_basin_daily.pcl'%(calname,calname,idx))
    tmp.to_hdf('/Volumes/data/RHESSys_out/Vcm/%s/%s_%s_basin_daily.hdf'%(calname,calname,idx),format='f',mode='w', \
               key='df',complevel=9,complib='bzip2',fletcher32=True)
    
    return None

In [39]:
res = view.map(process_cal,files)

In [40]:
p = pb.ProgressBar(len(files))

In [41]:
while res.ready() == False:
    p.animate_ipython(res.progress)
    time.sleep(2)

[****************100%******************]  49962 of 50000 complete


In [29]:
res.ready()

True